<a href="https://colab.research.google.com/github/ysMarcos/projeto-python/blob/main/projeto_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports das Bibliotecas usadas

In [16]:
#Imports
import requests as req
import hashlib
import pandas as pd
import sqlite3
import json
from google.colab import userdata

# Funções utilizadas

In [17]:
#Funcao responsavel para encryptografar para md5 de acordo com a documentacao da api da Marvel
def hashToMD5(publicKey, privateKey):
  #Valor 1 eh o timestamp de exemplo usado na documentacao
  return hashlib.md5(str.encode(str(1) + str(privateKey) + str(publicKey)))

In [18]:
# Funcao responsavel pelas requisicoes
def get(url: str, endpoint: str, headers: dict, params: dict):
  try:
    response = req.get(
        url = url+endpoint,
        headers = headers,
        params = params
      )
    return response.json()
  except Exception as e:
    print(e)

In [19]:
#Funcoes responsaveis pela transformacao dos dados em um Dataframe e salvar o objeto passado
def saveToCSV(obj, csvPath):
  df = pd.DataFrame(obj)
  df.to_csv(csvPath + '.csv', encoding = 'utf-8', index = False, header = True)

def saveToSqlite(obj, tableName, dbPath='bd.db', uniqueColumns = None):
  try:
    df = pd.DataFrame(obj) if not isinstance(obj, pd.DataFrame) else obj
    with sqlite3.connect(dbPath) as con:
      df.to_sql(tableName, con, if_exists='append', index=False)
# Passei o trabalho pro banco remover as duplicatas em vez da aplicacao, ganhando performance
      if(uniqueColumns):
        uniqueClause = ", ".join(uniqueColumns)
        query = f"""
          DELETE FROM {tableName}
          WHERE rowid NOT IN (
              SELECT MIN(rowid)
              FROM {tableName}
              GROUP BY {uniqueClause}
          );
          """
        con.execute(query)
  except Exception as e:
    print(f"Erro: {e}")

# Endpoints

In [49]:
#Variaveis que serao utilizadas em todos os endpoints da marvel
url = 'http://gateway.marvel.com/v1/public/'
publicKey = userdata.get('publicKey')
privateKey = userdata.get('privateKey')
securityHash = hashToMD5(publicKey, privateKey).hexdigest()
dbPath = userdata.get('dbPath')

headers = {
    'Accept': "*/*"
}

params = {
    "ts": 1,
    "apikey": publicKey,
    "hash": securityHash,
    "limit": 100
}

## Characters

In [25]:
endpoint = 'characters'

result = get(url, endpoint, headers, params)['data']['results']
characters = [{
    "id": x['id'],
    "name": x["name"],
    "description": x["description"],
    "comics": ''.join([y["name"] for y in x["comics"]["items"]]),
    "events": ''.join([y["name"] for y in x["events"]["items"]])
  } for x in result]
saveToCSV(csvPath = endpoint, obj = characters)
saveToSqlite(characters, endpoint, dbPath, ['id'])

## Comics

In [48]:
endpoint = 'comics'

result = get(url, endpoint, headers, params)['data']['results']
comics = [{
    "id": x["id"],
    "digitalId": x["digitalId"],
    "title": x["title"],
    "pageCount": x["pageCount"],
    "issueNumber": x["issueNumber"],
    "dateType": ''.join([y["type"] for y in x["dates"]]),
    "dates": ''.join([y["date"] for y in x["dates"]]),
    "priceType": ''.join([y["type"] for y in x["prices"]]),
    "prices": ''.join([str(y["price"]) for y in x["prices"]]),
    "events": ''.join([y["name"] for y in x["events"]["items"]]),
    "creators": ''.join(x["creators"])
  } for x in result]
saveToCSV(csvPath = endpoint, obj = comics)
saveToSqlite(comics, endpoint, dbPath, ["id"])

## Events

In [50]:
endpoint = "events"

result = get(url, endpoint, headers, params)["data"]["results"]
events = [{
    "id": x["id"],
    "title": x["title"],
    "description": x["description"],
    "start": x["start"],
    "end": x["end"],
    "comics": ''.join([y["name"] for y in x["comics"]["items"]]),
    "characters": ''.join([y["name"] for y in x["characters"]["items"]])
} for x in result]
saveToCSV(csvPath = endpoint, obj = events)
saveToSqlite(events, endpoint, dbPath, ["id"])